# Advanced Machine Learning, MScA (32017)

# Week 2: Homework Assignment

# Programming ANOVA in Spark with MAP-REDUCE

## Yuri Balasanov, Mikhail Tselishchev, &copy; iLykey, 2017 

## ANOVA Data and Problem Statement

Start PySpark notebook.
Check Spark settings

In [1]:
spark

Read data from file `data.csv` located in the same folder as the notebook.

In [3]:
raw_data = sc.textFile('./data1.csv')
raw_data.take(10)

['1.08931931750671 "C2"',
 '4.03892811665213 "C4"',
 '0.710538426311777 "C1"',
 '4.54878758817833 "C4"',
 '3.54970766958625 "C3"',
 '3.62544730346494 "C2"',
 '1.10997285637598 "C3"',
 '0.768292941585903 "C2"',
 '4.10632061985913 "C4"',
 '2.79791643753108 "C2"']

Each row of the dataset contains a response number and class name.
Such data are analyzed by ANOVA. <br> 
The goal is to calculate F statistic and the corresponding p-value to test null hypothesis $H_0:$ "All group mean values are equal".

### Group Totals

Create an RDD containing class name and a tuple of response value and combinator indicator 1 in the form:

[('C2', (1.08931931750671, 1)), <br>
 ('C4', (4.03892811665213, 1)), <br>
 ('C1', (0.710538426311777, 1)), <br>
 ('C4', (4.54878758817833, 1)), <br>
 ('C3', (3.54970766958625, 1))] <br>
 
 Incert code in the following cell. <br>
 **Hint**. Use `map()`

In [4]:
raw_data_split = raw_data.map(lambda x:x.split(" ")).map(lambda x: (x[1][1:-1], float(x[0]),int(1)))
raw_data_split = raw_data_split.map(lambda x: (x[0], (x[1],x[2])))
raw_data_split.collect()

[('C2', (1.08931931750671, 1)),
 ('C4', (4.03892811665213, 1)),
 ('C1', (0.710538426311777, 1)),
 ('C4', (4.54878758817833, 1)),
 ('C3', (3.54970766958625, 1)),
 ('C2', (3.62544730346494, 1)),
 ('C3', (1.10997285637598, 1)),
 ('C2', (0.768292941585903, 1)),
 ('C4', (4.10632061985913, 1)),
 ('C2', (2.79791643753108, 1)),
 ('C1', (-0.147657009236351, 1)),
 ('C2', (1.04310811867438, 1)),
 ('C2', (3.47025699708299, 1)),
 ('C2', (1.13021712631351, 1)),
 ('C4', (2.27934933792281, 1)),
 ('C2', (1.5518257633966, 1)),
 ('C3', (2.49862168220491, 1)),
 ('C4', (4.07734227692233, 1)),
 ('C4', (3.93353322592249, 1)),
 ('C1', (1.00837095999603, 1)),
 ('C3', (2.55661519563607, 1)),
 ('C4', (7.26641451887544, 1)),
 ('C4', (3.02544753128557, 1)),
 ('C3', (3.9514984696255, 1)),
 ('C4', (3.76161304994023, 1)),
 ('C4', (5.00736436564955, 1)),
 ('C4', (3.59501284151563, 1)),
 ('C1', (1.77214218580453, 1)),
 ('C4', (3.93098270250183, 1)),
 ('C2', (2.02747533674519, 1)),
 ('C3', (2.5790021021833, 1)),
 ('C3',

Create dictionary `counts` containing class name as key and a tuple of sum of class responses together with the class count as value.
The form of the dictionary is

{'C1': (102.26684498157968, 100), <br>
 'C2': (195.44484415126036, 100), <br>
 'C3': (306.7221289432121, 100), <br>
 'C4': (400.3225323072952, 100)} <br>
 
 Using dictionary format rather than RDD for group totals and counts is appropriate because it is a small object.
 
 Insert code in the following cell. <br>
 **Hint**. This can be done by `reduceByKey()`, `sortByKey()` and `collectAsMap()`

In [5]:
counts = raw_data_split.reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1])).collectAsMap()
counts

{'C1': (102.26684498157968, 100),
 'C2': (195.44484415126036, 100),
 'C3': (306.7221289432121, 100),
 'C4': (400.3225323072952, 100)}

### Group Means and Grand Mean

Calculate number of classes `nClasses` and total number of observations `nObs`.

Enter code computing `nClasses` and `nObs` in the following cell.

In [6]:
nClasses = len(counts)
nObs = nClasses * counts['C1'][1]
print(nClasses, nObs)


4 400


Calculate group mean values and add them to dictionary `counts`. <br>
Resulting dictionary `counts_m` should loo like this: <br>

{'C1': (102.26684498157968, 1.0226684498157967, 100), <br>
 'C2': (195.44484415126036, 1.9544484415126036, 100), <br>
 'C3': (306.7221289432121, 3.067221289432121, 100), <br>
 'C4': (400.3225323072952, 4.003225323072952, 100)} <br>
 
 Value of the dictionary contains group total, group mean and group count.
 
 Enter code calculating `counts_m` in the following cell.

In [7]:
counts_m = {k: (v[0], v[0] / v[1], v[1]) for k, v in counts.items()} # dictionary comprehension, dict.keys(), .values()  .items()
counts_m

{'C1': (102.26684498157968, 1.0226684498157967, 100),
 'C2': (195.44484415126036, 1.9544484415126036, 100),
 'C3': (306.7221289432121, 3.067221289432121, 100),
 'C4': (400.3225323072952, 4.003225323072952, 100)}

Calculate `grandMean` using `map()` and `mean()` functions.
Insert necessary code in the following cell.

In [8]:
grandMean = raw_data_split.map(lambda x: (x[1][0])).mean()
grandMean


2.5118908759583674

### Between and Within Groups Sums of Squares

Calclulate sums of squares `betweenSS` and `withinSS`, their degrees of freedom `dfClass` and `dfResid` and mean square values
$$meanSqClass=\frac{betweenSS}{dfClass},~ meanSqResid=\frac{withinSS}{dfResid}.$$
Enter code in the following 2 cells.

In [9]:
#meanSqClass
betweenSS = sum([(v[1]-grandMean)**2 for v in counts_m.values()])
dfClass = nClasses -1
meanSqClass = betweenSS/dfClass
print(meanSqClass)

1.6869986011754259


In [41]:
#meanSqResid
withinSSb=raw_data_split.map(lambda r: (r[1][0]-counts_m[r[0]][1])**2).collect()
withinSS = sum(withinSSb)
dfResid = nObs - 1
meanSqResid = withinSS/dfResid
meanSqResid

0.9211426924602995

In [42]:
#f_statistic
f_stat=meanSqClass/meanSqResid
f_stat


1.8314194043808627

In [43]:
#p-value
from scipy.stats import f
f.cdf(f_stat,dfClass ,dfResid)


0.85916778936971772